<h3>After data analysis

In [1]:
import pandas as pd
data=pd.read_csv("train.csv")

In [2]:
data.head()

,MWG,NWG,KWG,MDIMC,NDIMC,MDIMA,NDIMB,KWI,VWM,VWN,STRM,STRN,SA,SB,Run1 (ms),Run2 (ms),Run3 (ms),Run4 (ms)
0,128,128,16,16,16,16,16,2,1,1,0,0,0,0,287.87,288.66,289.45,288.24
1,128,128,16,16,16,16,16,2,1,1,0,0,0,1,241.13,247.84,248.27,247.49
2,128,128,16,16,16,16,16,2,1,1,0,0,1,1,148.12,149.99,149.90,150.14
3,128,128,16,16,16,16,16,2,1,1,0,1,0,0,301.40,302.43,302.70,301.94
4,128,128,16,16,16,16,16,2,1,1,0,1,1,0,244.74,251.80,252.15,251.04


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 18 columns):
MWG          200000 non-null int64
NWG          200000 non-null int64
KWG          200000 non-null int64
MDIMC        200000 non-null int64
NDIMC        200000 non-null int64
MDIMA        200000 non-null int64
NDIMB        200000 non-null int64
KWI          200000 non-null int64
VWM          200000 non-null int64
VWN          200000 non-null int64
STRM         200000 non-null int64
STRN         200000 non-null int64
SA           200000 non-null int64
SB           200000 non-null int64
Run1 (ms)    200000 non-null float64
Run2 (ms)    200000 non-null float64
Run3 (ms)    200000 non-null float64
Run4 (ms)    200000 non-null float64
dtypes: float64(4), int64(14)
memory usage: 27.5 MB


In [4]:
features=data.drop(["Run1 (ms)","Run2 (ms)","Run3 (ms)","Run4 (ms)"],axis="columns")

In [5]:
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 14 columns):
MWG      200000 non-null int64
NWG      200000 non-null int64
KWG      200000 non-null int64
MDIMC    200000 non-null int64
NDIMC    200000 non-null int64
MDIMA    200000 non-null int64
NDIMB    200000 non-null int64
KWI      200000 non-null int64
VWM      200000 non-null int64
VWN      200000 non-null int64
STRM     200000 non-null int64
STRN     200000 non-null int64
SA       200000 non-null int64
SB       200000 non-null int64
dtypes: int64(14)
memory usage: 21.4 MB


In [10]:
features.nunique()

MWG      4
NWG      4
KWG      2
MDIMC    3
NDIMC    3
MDIMA    3
NDIMB    3
KWI      2
VWM      4
VWN      4
STRM     2
STRN     2
SA       2
SB       2
dtype: int64

In [11]:
labels=data[["Run1 (ms)","Run2 (ms)","Run3 (ms)","Run4 (ms)"]]

<h3>Applied train test split

In [12]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(features,labels,random_state=42,test_size=.33)

<h4>Applied xgBoost on train test data

In [13]:
import xgboost as xgb
reg = xgb.XGBRegressor(max_depth=15, n_estimators=100, learning_rate=0.1)
reg.fit(x_train,y_train["Run1 (ms)"],eval_metric="error",verbose=True)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=15, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [14]:
pred = reg.predict(x_test)
from sklearn.metrics import mean_squared_error
mean_squared_error(pred,y_test["Run1 (ms)"])

<h2>Applying AdaBoostRegressor

In [17]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor

In [18]:
reg = AdaBoostRegressor(DecisionTreeRegressor(max_depth=30),n_estimators=100,loss='exponential')

In [19]:
reg.fit(features,labels["Run1 (ms)"])

AdaBoostRegressor(base_estimator=DecisionTreeRegressor(criterion='mse', max_depth=30, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best'),
         learning_rate=1.0, loss='exponential', n_estimators=100,
         random_state=None)

In [20]:
pred = reg.predict(x_test)
mean_squared_error(pred,y_test["Run1 (ms)"])

0.0038718287878787717

In [26]:
data_test=pd.read_csv('test.csv')
data_test.drop("Id",axis=1,inplace=True)
data_test.info()
ans={"Run1 (ms)":[],"Run2 (ms)":[],"Run3 (ms)":[],"Run4 (ms)":[]}
ans["Run1 (ms)"]=reg.predict(data_test)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41601 entries, 0 to 41600
Data columns (total 14 columns):
MWG      41601 non-null int64
NWG      41601 non-null int64
KWG      41601 non-null int64
MDIMC    41601 non-null int64
NDIMC    41601 non-null int64
MDIMA    41601 non-null int64
NDIMB    41601 non-null int64
KWI      41601 non-null int64
VWM      41601 non-null int64
VWN      41601 non-null int64
STRM     41601 non-null int64
STRN     41601 non-null int64
SA       41601 non-null int64
SB       41601 non-null int64
dtypes: int64(14)
memory usage: 4.4 MB


In [27]:
reg.fit(features,labels["Run2 (ms)"])
ans["Run2 (ms)"]=reg.predict(data_test)
reg.fit(features,labels["Run3 (ms)"])
ans["Run3 (ms)"]=reg.predict(data_test)
reg.fit(features,labels["Run4 (ms)"])
ans["Run4 (ms)"]=reg.predict(data_test)
data_test = pd.read_csv("test.csv")
ans["Id"]=data_test["Id"]
sub = pd.DataFrame(ans)
sub.to_csv("sampleSubmission.csv")